## Match Training set to LongHealth

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-Distractor")
# model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-Distractor")

# context = r"""
# World number one Novak Djokovic says he is hoping for a "positive decision" to allow him
# to play at Indian Wells and the Miami Open next month. The United States has extended
# its requirement for international visitors to be vaccinated against Covid-19. Proof of vaccination
# will be required to enter the country until at least 10 April, but the Serbian has previously
# said he is unvaccinated. The 35-year-old has applied for special permission to enter the country.
# Indian Wells and the Miami Open - two of the most prestigious tournaments on the tennis calendar
# outside the Grand Slams - start on 6 and 20 March respectively. Djokovic says he will return to
# the ATP tour in Dubai next week after claiming a record-extending 10th Australian Open title
# and a record-equalling 22nd Grand Slam men's title last month.""".replace("\n", "")
# question = "What is the best title for the passage?"
# answer = "Djokovic's application for special permission to enter the United States"

# input_text = " ".join([question, tokenizer.sep_token, answer, tokenizer.sep_token, context])
# inputs = tokenizer(input_text, return_tensors="pt")
# outputs = model.generate(**inputs, max_new_tokens=128)
# distractors = tokenizer.decode(outputs[0], skip_special_tokens=False)
# distractors = distractors.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
# distractors = [y.strip() for y in distractors.split(tokenizer.sep_token)]
# print(distractors)

In [3]:
# context

In [4]:
import json
import pandas as pd

with open('medication-SQUAD.json') as f1:
    data1 = json.load(f1)
df1 = pd.DataFrame(data1)

print("First 10 rows of file1.json:")
print(df1.head(10))

# Print the size of each DataFrame
print("\nSize of DataFrame from file1.json:")
print(df1.size)


training_data = []

for _, row in df1.iterrows():
    data = row.iloc[0]  # Access the nested structure
    for paragraph in data['paragraphs']:
        for qa in paragraph['qas']:
            qa_dict = {}
            qa_dict['title'] = data['title']
            qa_dict['context'] = paragraph['context']
            qa_dict['question'] = qa['question']
            qa_dict['id'] = qa['id']
            qa_dict['answer_start'] = qa['answers'][0]['answer_start']
            qa_dict['text'] = qa['answers'][0]['text']
            training_data.append(qa_dict)

df_training = pd.DataFrame(training_data)

First 10 rows of file1.json:
                                                data  version
0  {'title': 'RECORD #91996', 'paragraphs': [{'co...      1.0
1  {'title': 'RECORD #5091', 'paragraphs': [{'con...      1.0
2  {'title': 'RECORD #230210', 'paragraphs': [{'c...      1.0
3  {'title': 'RECORD #311514', 'paragraphs': [{'c...      1.0
4  {'title': 'RECORD #701307', 'paragraphs': [{'c...      1.0
5  {'title': 'RECORD #718365', 'paragraphs': [{'c...      1.0
6  {'title': 'RECORD #293045', 'paragraphs': [{'c...      1.0
7  {'title': 'RECORD #874479', 'paragraphs': [{'c...      1.0
8  {'title': 'RECORD #831462', 'paragraphs': [{'c...      1.0
9  {'title': 'RECORD #214865', 'paragraphs': [{'c...      1.0

Size of DataFrame from file1.json:
522


In [5]:
# df_training = df_training[['context', 'question', 'text']]
# df_training = df_training.rename(columns={'text': 'correct'})

In [6]:
# df_training = df_training.head(10)

In [7]:
# df_training

In [8]:
# import torch
# import random
# from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GPT2LMHeadModel, GPT2Tokenizer

# tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-Distractor")
# model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-Distractor")

# gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# model = model.to(device)
# gpt2_model = gpt2_model.to(device)

# new_columns = ['answer_a', 'answer_b', 'answer_c', 'answer_d', 'answer_e']

# for column in new_columns:
#     df_training[column] = ""

# for index, row in df_training.iterrows():
#     # Use the model to generate distractors
#     context = row['context']
#     question = row['question']
#     answer = row['correct']
#     input_text = " ".join([question, tokenizer.sep_token, answer, tokenizer.sep_token, context])
#     inputs = tokenizer(input_text, return_tensors="pt").to(device)

#     outputs = model.generate(**inputs, max_new_tokens=128)
#     distractors = tokenizer.decode(outputs[0], skip_special_tokens=False)
#     distractors = distractors.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
#     distractors = [y.strip() for y in distractors.split(tokenizer.sep_token)]

#     distractor_to_expand = random.choice(distractors)

#     gpt2_inputs = gpt2_tokenizer.encode(distractor_to_expand, return_tensors="pt").to(device)
#     gpt2_outputs = gpt2_model.generate(gpt2_inputs, max_length=50, do_sample=True)
#     similar_sentence = gpt2_tokenizer.decode(gpt2_outputs[0], skip_special_tokens=True)

#     distractors.append(similar_sentence)

#     choices = [answer] + distractors

#     random.shuffle(choices)

#     for i, column in enumerate(new_columns):
#         df_training.at[index, column] = choices[i]

# new_column_order = ['context', 'question'] + new_columns + ['correct']

# df_training = df_training[new_column_order]

In [9]:
# df_training

In [10]:
# Without title id answer start
# paragraphs_dict = {}

# qa_pairs = []

# for _, row in df1.iterrows():
#     data = row.iloc[0]
#     for paragraph in data['paragraphs']:
#         if paragraph['context'] not in paragraphs_dict:
#             paragraphs_dict[paragraph['context']] = len(paragraphs_dict)

#         paragraph_id = paragraphs_dict[paragraph['context']]

#         for qa in paragraph['qas']:
#             qa_dict = {}
#             qa_dict['paragraph_id'] = paragraph_id
#             qa_dict['question'] = qa['question']
#             qa_dict['correct'] = qa['answers'][0]['text']
#             qa_pairs.append(qa_dict)

# df_paragraphs = pd.DataFrame(list(paragraphs_dict.items()), columns=['context', 'paragraph_id'])
# df_qa_pairs = pd.DataFrame(qa_pairs)

In [11]:
# all data kept
# Initialize a dictionary to store unique paragraphs
paragraphs_dict = {}

# Initialize a list to store QA pairs
qa_pairs = []

# Iterate over each row in df1
for _, row in df1.iterrows():
    data = row.iloc[0]  # Access the nested structure
    for paragraph in data['paragraphs']:
        # Store each unique paragraph in paragraphs_dict
        # Use the paragraph's context as the key and assign it a unique ID
        if paragraph['context'] not in paragraphs_dict:
            paragraphs_dict[paragraph['context']] = len(paragraphs_dict)

        # Get the ID of the paragraph
        paragraph_id = paragraphs_dict[paragraph['context']]

        for qa in paragraph['qas']:
            # Store each QA pair along with the ID of its paragraph
            qa_dict = {}
            qa_dict['title'] = data['title']
            qa_dict['paragraph_id'] = paragraph_id
            qa_dict['question'] = qa['question']
            qa_dict['id'] = qa['id']
            qa_dict['answer_start'] = qa['answers'][0]['answer_start']
            qa_dict['correct'] = qa['answers'][0]['text']
            qa_pairs.append(qa_dict)

# Convert paragraphs_dict and qa_pairs to DataFrames
df_paragraphs = pd.DataFrame(list(paragraphs_dict.items()), columns=['context', 'paragraph_id'])
df_qa_pairs = pd.DataFrame(qa_pairs)

In [12]:
df_qa_pairs

,title,paragraph_id,question,id,answer_start,correct
0,RECORD #91996,0,Was the patient ever prescribed aspirin,2a02170ee63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
1,RECORD #91996,0,Has this patient ever been on aspirin,2a021a4ce63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
2,RECORD #91996,0,Is the patient currently or have they ever tak...,2a021a9ce63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
3,RECORD #91996,0,has there been a prior aspirin,2a021ac4e63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
4,RECORD #91996,0,Has this patient ever tried aspirin,2a021ae2e63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
...,...,...,...,...,...,...
222952,RECORD #210958,259,Why was the patient on sarna,2af94ddae63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash
222953,RECORD #210958,259,Was the patient ever given sarna for bilateral...,2af94ea2e63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash
222954,RECORD #210958,259,Did the patient receive sarna for bilateral lo...,2af94ec0e63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash
222955,RECORD #210958,259,Why did the patient need sarna,2af94f56e63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash


In [13]:
# df_qa_pairs = df_qa_pairs.head(3)

In [14]:
df_qa_pairs

,title,paragraph_id,question,id,answer_start,correct
0,RECORD #91996,0,Was the patient ever prescribed aspirin,2a02170ee63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
1,RECORD #91996,0,Has this patient ever been on aspirin,2a021a4ce63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
2,RECORD #91996,0,Is the patient currently or have they ever tak...,2a021a9ce63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
3,RECORD #91996,0,has there been a prior aspirin,2a021ac4e63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
4,RECORD #91996,0,Has this patient ever tried aspirin,2a021ae2e63211ee822dacde48001122,545,"no relief with Tylenol, aspirin or Fioricet. S..."
...,...,...,...,...,...,...
222952,RECORD #210958,259,Why was the patient on sarna,2af94ddae63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash
222953,RECORD #210958,259,Was the patient ever given sarna for bilateral...,2af94ea2e63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash
222954,RECORD #210958,259,Did the patient receive sarna for bilateral lo...,2af94ec0e63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash
222955,RECORD #210958,259,Why did the patient need sarna,2af94f56e63211ee822dacde48001122,799,Instructions: for bilateral lower extremity rash


In [ ]:
import torch
import random
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, GPT2LMHeadModel, GPT2Tokenizer

tokenizer = AutoTokenizer.from_pretrained("potsawee/t5-large-generation-race-Distractor")
model = AutoModelForSeq2SeqLM.from_pretrained("potsawee/t5-large-generation-race-Distractor")

gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = model.to(device)
gpt2_model = gpt2_model.to(device)

new_columns = ['answer_a', 'answer_b', 'answer_c', 'answer_d', 'answer_e']

for column in new_columns:
    df_qa_pairs[column] = ""

counter = 0

for index, row in df_qa_pairs.iterrows():
    # Use the model to generate distractors
    print(counter)
    context = df_paragraphs.loc[df_paragraphs['paragraph_id'] == row['paragraph_id'], 'context'].values[0]
    question = row['question']
    answer = row['correct']
    input_text = " ".join([question, tokenizer.sep_token, answer, tokenizer.sep_token, context])
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # Generate distractors
    outputs = model.generate(**inputs, max_new_tokens=128)
    distractors = tokenizer.decode(outputs[0], skip_special_tokens=False)
    distractors = distractors.replace(tokenizer.pad_token, "").replace(tokenizer.eos_token, "")
    distractors = [y.strip() for y in distractors.split(tokenizer.sep_token)]
    distractor_to_expand = random.choice(distractors)

    gpt2_inputs = gpt2_tokenizer.encode(distractor_to_expand, return_tensors="pt").to(device)
    gpt2_outputs = gpt2_model.generate(gpt2_inputs, max_length=50, do_sample=True)
    similar_sentence = gpt2_tokenizer.decode(gpt2_outputs[0], skip_special_tokens=True)

    distractors.append(similar_sentence)

    choices = [answer] + distractors

    random.shuffle(choices)
    counter = counter + 1
    for i, column in enumerate(new_columns):
        df_qa_pairs.at[index, column] = choices[i]

new_column_order = ['paragraph_id', 'id', 'question'] + new_columns + ['correct', 'answer_start']
df_qa_pairs = df_qa_pairs[new_column_order]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

...

286


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


287


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


288


In [ ]:
df_qa_pairs

In [ ]:
df_paragraphs

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Save df_paragraphs to a CSV file
df_paragraphs.to_csv('/content/drive/My Drive/df_paragraphs.csv', index=False)

# Save df_qa_pairs to a CSV file
df_qa_pairs.to_csv('/content/drive/My Drive/df_qa_pairs.csv', index=False)


In [ ]:
data_dict = {}

for index, row in df_qa_pairs.iterrows():
    # Get the paragraph id
    paragraph_id = row['paragraph_id']

    if paragraph_id not in data_dict:
        data_dict[paragraph_id] = {
            'paragraph': df_paragraphs.loc[df_paragraphs['paragraph_id'] == paragraph_id, 'context'].values[0],
            'questions': []
        }

    question_dict = {
        'id': row['id'],
        'question': row['question'],
        'answer_a': row['answer_a'],
        'answer_b': row['answer_b'],
        'answer_c': row['answer_c'],
        'answer_d': row['answer_d'],
        'answer_e': row['answer_e'],
        'correct': row['correct'],
        'answer_start': row['answer_start']
    }

    data_dict[paragraph_id]['questions'].append(question_dict)

json_data = json.dumps(data_dict)

# If you want to write this JSON data to a file, you can do so like this:
with open('/content/drive/My Drive/data.json', 'w') as f:
    f.write(json_data)


In [ ]:
json_data